## Tarun Nadipalli - 705.603 Creating AI-Enabled Systems Final Project

For my final project, I decided to build my own content-based song recommendation application with the Spotify API. Users are able to input a link to a Spotify playlist that contains songs they like and my application will send back a link to playlist that contains my recommended songs. This guide focuses on how Spotify's Million Playlist Dataset was used to collect hundreds of thousangs of songs that will act as the pool of potential recommendations.

### Data Collection Module (`data.py`) Guide

This Jupyter notebook outlines the functionality built in the `data.py` script that allows us to collect millions of unique song ID's from Spotify's Million Playlist Dataset and then sequentially use the Spotify API to gather audio feature data for all of those songs. This script is what was used to collect all of the data necessary for the recommendation model. 

#### Spotify Million Playlist Dataset
To begin, we must first discuss the Spotify Million Playlist Dataset, and how / why we are using it.

**Background**

As part of RecSys 2018 (annual Conference on Recommender Systems), Spotify released a dataset that contains 1 million user created playlists (made between 2010 and 2017) sampled from their 4 billion total playlists at the time. They released this dataset for the following challenge: Given a seed playlist title and initial set of tracks in a playlist, predict the subsequent tracks in a playlist. 

**Dataset Structure**

The Spotify Million Playlist Dataset (MPD) is sampled from 4 billion public playlists on Spotify, contains 1 million playlists that have over 2 million unique songs and 300,000 unique artists. Each playlist in MPD contains playlist title, track list of IDs, and other metadata about the playlist. Note that:

> "Playlists are sampled with some randomization, are manually filtered for playlist quality and to remove offensive content, and have some dithering and fictitious tracks added to them. As such, the dataset is not representative of the true distribution of playlists on the Spotify platform, and must not be interpreted as such in any research or analysis performed on the dataset." per [AICrowd](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge)


<details>
<summary>Here is an example of a playlist json object in the MPD:</summary>

``` json
{
        "name": "musical",
        "collaborative": "false",
        "pid": 5,
        "modified_at": 1493424000,
        "num_albums": 7,
        "num_tracks": 12,
        "num_followers": 1,
        "num_edits": 2,
        "duration_ms": 2657366,
        "num_artists": 6,
        "tracks": [
            {
                "pos": 0,
                "artist_name": "Degiheugi",
                "track_uri": "spotify:track:7vqa3sDmtEaVJ2gcvxtRID",
                "artist_uri": "spotify:artist:3V2paBXEoZIAhfZRJmo2jL",
                "track_name": "Finalement",
                "album_uri": "spotify:album:2KrRMJ9z7Xjoz1Az4O6UML",
                "duration_ms": 166264,
                "album_name": "Dancing Chords and Fireflies"
            },
            {
                "pos": 1,
                "artist_name": "Degiheugi",
                "track_uri": "spotify:track:23EOmJivOZ88WJPUbIPjh6",
                "artist_uri": "spotify:artist:3V2paBXEoZIAhfZRJmo2jL",
                "track_name": "Betty",
                "album_uri": "spotify:album:3lUSlvjUoHNA8IkNTqURqd",
                "duration_ms": 235534,
                "album_name": "Endless Smile"
            },
            {
                "pos": 2,
                "artist_name": "Degiheugi",
                "track_uri": "spotify:track:1vaffTCJxkyqeJY7zF9a55",
                "artist_uri": "spotify:artist:3V2paBXEoZIAhfZRJmo2jL",
                "track_name": "Some Beat in My Head",
                "album_uri": "spotify:album:2KrRMJ9z7Xjoz1Az4O6UML",
                "duration_ms": 268050,
                "album_name": "Dancing Chords and Fireflies"
            },
            // 8 tracks omitted
            {
                "pos": 11,
                "artist_name": "Mo' Horizons",
                "track_uri": "spotify:track:7iwx00eBzeSSSy6xfESyWN",
                "artist_uri": "spotify:artist:3tuX54dqgS8LsGUvNzgrpP",
                "track_name": "Fever 99\u00b0",
                "album_uri": "spotify:album:2Fg1t2tyOSGWkVYHlFfXVf",
                "duration_ms": 364320,
                "album_name": "Come Touch The Sun"
            }
        ],

    }
```
</details>

All of the million playlists are contained in over 1,000 json files (slices), and each json file contains 1,000 playlists. Our effort with this script is to iterate over these files and playlists and extract all of the unique songs and their audio features.

**Why**

The reason I chose to use this dataset as a pool of songs to recommend from is for a few different reasons. 
1. This is a clean dataset that already contains around 2 million unique song IDs. I wouldn't have to look elsewhere online or find large playlists in Spotify myself. I could never personally achieve that large of a dataset within the time frame of this project.
   
2. Having the song IDs already saves me tons of resources in terms of read / writes from the Spotify API to my local database.
3. As per the quote above from AICrowd, this dataset is filtered to have songs of quality. Anyone, after all, can upload songs to Spotify, and although they may be good songs, I only want to recommend songs that are validated by others (since users actively added these to playlists) to ensure cleanliness of the songs I recommend.
4. As a follow up to #3, this dataset is by no means representative of the true distribution of songs in Spotify. However, this project is more so about the recommendations, rather than the pool of songs. 

### Methods

Before I begin describing the methods and their functionality, it must be noted that some of the methods are based off of the work done be @nsanka on Github [here](https://github.com/nsanka/RecSys/blob/main/code/read_spotify_million_playlists.py). This person found intelligent ways to speed up the processing of this large dataset for which my Macbook is thankful (especially since Google Chrome takes up all my RAM).

Note: As this is a guide, I will only be showing simple examples of how these methods work. For the actual project data collection, I have run the entire script locally and collected 300,000 songs to work upon (2 million was too much for my computer)

##### create_connection(db_file)
The first method we will look at is `create_connection(db_file)` which allows us to connect to a SQLite local file database. SQLite is a self-contained file-based SQL database that comes bundled with Python and has greater read / write efficiency compared to a .csv file or .json file. Here we instantiate a .db file that SQLite can then create schemas, tables and store data within. 

In [1]:
from data import *

zip_file = './data/spotify_million_playlist_dataset.zip'
db_file = './temp.db'

conn = create_connection(db_file)
print(conn)

INFO:root:Connection to ./temp.db is successful!


#### create_table(conn, create_table_sql, table_name)

The second method lets us use the SQLite connection object we previously made to connect to the temp.db file and create tables within. For our use case, we will create two tables: tracks and features. The tracks table will contain all of the unique song IDs and the features table will contain the audio features from Spotify for each of those song IDs. 

This method takes in three parameters: SQLite connection object, the table name string and the create_table_sql query string. This query string is a SQL query that defines the table schema for us so we can instantiate the table properly. 

The create_table_sql query is passed in through the next method:

#### create_all_tables(conn)

This function uses the `create_table(conn, create_table_sql, table_name)` method to create the tracks and features tables in our temp.db file. The create_table_sql queries are as below:

<details>
<summary>Create Table SQL Query for Tracks table:</summary>

``` mysql

CREATE TABLE IF NOT EXISTS tracks (
track_uri text NOT NULL,
track_id integer NOT NULL
);

```
</details>

<details>
<summary>Create Table SQL Query for Features table:</summary>

``` mysql

CREATE TABLE IF NOT EXISTS features (
track_id integer,
track_uri text NOT NULL,
danceability real,
energy real,
key real,
loudness real,
mode real,
speechiness real,
acousticness real,
instrumentalness real,
liveness real,
valence real,
tempo real,
duration_ms integer,
time_signature integer
);

```
</details>

Now, let's go ahead and instantiate the tables in our temp.db SQLite database. 

In [2]:
create_all_tables(conn)

INFO:root:Created table tracks successfully!
INFO:root:Created table features successfully!


To show that this works, let's run our own query using the `conn` object on our tables to the schema of the tables we've made. As we can see below, it works!

In [3]:
cur = conn.cursor()
query = cur.execute(
"""
PRAGMA table_info('tracks');   
""" 
)
results = query.fetchall()
print("Tracks Table Schema: ", results)

query = cur.execute(
"""
PRAGMA table_info('features');   
""" 
)
results = query.fetchall()
print("Features Table Schema: ", results)

Tracks Table Schema:  [(0, 'track_uri', 'TEXT', 1, None, 0), (1, 'track_id', 'INTEGER', 1, None, 0)]
Features Table Schema:  [(0, 'track_id', 'INTEGER', 0, None, 0), (1, 'track_uri', 'TEXT', 1, None, 0), (2, 'danceability', 'REAL', 0, None, 0), (3, 'energy', 'REAL', 0, None, 0), (4, 'key', 'REAL', 0, None, 0), (5, 'loudness', 'REAL', 0, None, 0), (6, 'mode', 'REAL', 0, None, 0), (7, 'speechiness', 'REAL', 0, None, 0), (8, 'acousticness', 'REAL', 0, None, 0), (9, 'instrumentalness', 'REAL', 0, None, 0), (10, 'liveness', 'REAL', 0, None, 0), (11, 'valence', 'REAL', 0, None, 0), (12, 'tempo', 'REAL', 0, None, 0), (13, 'duration_ms', 'INTEGER', 0, None, 0), (14, 'time_signature', 'INTEGER', 0, None, 0)]


#### extract_mpd_dataset(zip_file, num_files)

This method was adapted from @nsanka on GitHub. As mentioned previously, the MPD dataset is a zip file of 1000 json files. The most efficient time/storage method of extracting the json data from these files is to use the zipfile, fnmatch, and json libraries. With these modules, we can iterate through all the files in the zip, find only the json files, and dump all of the json data into our database or process it from there. The best part of this solution is that we do not need to unzip the MPD zip file to get all of the data, greatly reducing the storage needed!

The parameters for this function are the path to the MPD zip file and the number of files we would like to process out of the 1000. This function doesn't return anything and instead processes each json file iteratively - to show how it works I will run a modification of the code included in this function in the data.py file.

In [4]:
# we will test this function on the first three json files in MPD
def extract_mpd_dataset_modified(zip_file, db_file, num_files=3):
    with ZipFile(zip_file) as zipfiles:
        file_list = zipfiles.namelist()
        
        json_files = fnmatch.filter(file_list, "*.json")
        json_files = [f for i,f in sorted([(int(filename.split('.')[2].split('-')[0]), filename) for filename in json_files])]
        logging.info('Obtained all .json files from MPD.zip!')
        cnt = 0
        for filename in json_files:
            cnt += 1
            with zipfiles.open(filename) as json_file:
                json_data = json.loads(json_file.read())
                # process_json_data(json_data, db_file)
                # process json data directly here ^
                
                print("File Name: ",filename)
                # print the first track of the first playlist in each file
                print("Sample Data: ",json_data['playlists'][0]['tracks'][0], "\n")

            if (cnt == num_files) and (num_files > 0):
                break
            
extract_mpd_dataset_modified(zip_file, db_file)

INFO:root:Obtained all .json files from MPD.zip!


File Name:  data/mpd.slice.0-999.json
Sample Data:  {'pos': 0, 'artist_name': 'Missy Elliott', 'track_uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI', 'artist_uri': 'spotify:artist:2wIVse2owClT7go1WT98tk', 'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)', 'album_uri': 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K', 'duration_ms': 226863, 'album_name': 'The Cookbook'} 

File Name:  data/mpd.slice.1000-1999.json
Sample Data:  {'pos': 0, 'artist_name': 'Original Broadway Cast - The Little Mermaid', 'track_uri': 'spotify:track:5IbCV9Icebx8rR6wAp5hhP', 'artist_uri': 'spotify:artist:3TymzPhJTMyupk7P5xkahM', 'track_name': 'Fathoms Below - Broadway Cast Recording', 'album_uri': 'spotify:album:3ULJeOMgroG27dpn27MDfS', 'duration_ms': 154506, 'album_name': 'The Little Mermaid: Original Broadway Cast Recording'} 

File Name:  data/mpd.slice.2000-2999.json
Sample Data:  {'pos': 0, 'artist_name': 'The Jackson 5', 'track_uri': 'spotify:track:6cb0HzFQPN4BGADOmSzPCw', 'artist_uri': 'spotify:artist:2iE18

### process_json_data(json_data)

As we can see from the `extract_mpd_dataset()` method above, we iterate through all the files in the zip and process the json data using this function. Ultimately, this function retrieves all of the unique song IDs from the playlist and places them inside the 'tracks' table in our database. 

This function follows the following workflow:
1. Get all the song IDs passed in from the extract_mpd_dataset() function
2. Perform a left join on song IDs existing in the 'tracks' table, and the new song IDs obtained in #1
   - The left join lets us see which song IDs are unique, new, and can be added to the table
3. Create a temp column to determine new songs to be added to the table based on the left join in #2
4. Delete any duplicates among the new songs to be added
5. Drop any unnecessary data columns (only keeping the song ID and the song index for later)
6. Add the new songs to the 'tracks' table

Now, let's run the modified `extract_mpd_dataset()` function that will process the data and paste it in the 'tracks' table where we can see all of our unique song IDs!

In [5]:
extract_mpd_dataset(zip_file, db_file, 3)

cur = conn.cursor()
query = cur.execute(
"""
SELECT * FROM 'tracks' LIMIT 10;
""" 
)
results = query.fetchall()

# The second value in each object returned is the Track Index (not a count metric)
print("Tracks Table: ", results)

INFO:root:Obtained all .json files from MPD.zip!
INFO:root:Connection to ./temp.db is successful!
INFO:root:Number of tracks that already exist: 0
INFO:root:Total unique tracks: 34443
INFO:root:Adding tracks to database: 1 to 34443
INFO:root:Connection to ./temp.db is successful!
INFO:root:Number of tracks that already exist: 12734
INFO:root:Total unique tracks: 23441
INFO:root:Adding tracks to database: 34444 to 57884
INFO:root:Connection to ./temp.db is successful!
INFO:root:Number of tracks that already exist: 16258
INFO:root:Total unique tracks: 18623
INFO:root:Adding tracks to database: 57885 to 76507


Tracks Table:  [('0UaMYEvWZi0ZqiDOoHU3YI', 2209), ('6I9VzXrHxO9rA9A5euc8Ak', 27709), ('0WqIKmW4BTrj3eJFmnCKMv', 2383), ('1AWQoqb9bSvzTjaLralEkT', 5177), ('1lzr43nnXAijIGYnCT8M8H', 7784), ('0XUfyU2QviPAs6bxSpXYG4', 2437), ('68vgtRHr7iZHpzGpon6Jlo', 27044), ('3BxWKCI06eQ5Od8TY2JBeA', 13943), ('7H6ev70Weq6DdpZyyTmUXk', 32180), ('2PpruBYCo4H7WOBJ7Q2EwM', 10585)]


#### get_max_track_id(conn, table_name)

This is a simple helper function that returns the max track ID (meaning index) so that we can correctly input new data at the correct next empty index inside the table and keep track of how many songs we have in our database. For example, after running the process_json_data() function above, we can now see that the tracks table has a max track id of 76507. Meaning that there are 76507 unique song IDs in the tracks table.

In [6]:
print(get_max_track_id(conn, 'tracks'))

76507


#### create_audio_features(conn, cnt_uris, max_songs)

The last (and most important) function is `create_audio_features()`. This method takes all of unique song IDs in the tracks table and queries the Spotify API for the songs' audio features. This method is pretty simple in that it takes 100 song IDs at a time, requests the Spotify API for their features, removes null responses, and adds the important columns to the 'features' table in the database. The features we are keeping are as follows:
Danceability, Energy, Key, Loudness, Mode, Speechiness, Acousticness, Instrumentalness, Liveness, Valence, Tempo, Duration, and Time Signature. The full definitions and analysis of this data will be covered in the data_analysis.ipynb notebook in this project repository.

The parameters are the connection object, cnt_uris is how many song IDs we include in each Spotify API call, and the max_songs parameter is a limit as to how many songs we want to put into our features table. For purposes of this example, we will only put 100 song features in our 'features' table in the database.

In [7]:
conn = create_connection(db_file)
create_audio_features(conn, cnt_uris=100, max_songs=100)

conn = create_connection(db_file)
cur = conn.cursor()
query = cur.execute(
"""
SELECT * FROM 'features' LIMIT 10;
""" 
)
results = query.fetchall()
print("Features Table: ", results)

INFO:root:Connection to ./temp.db is successful!
INFO:root:Minimum Track ID in Features: 0 | Max Track ID in Tracks: 76507
INFO:root:Getting audio features for Track ID: 1 to 100
INFO:root:Retrieved track audio features!
INFO:root:Connection to ./temp.db is successful!


Features Table:  [(1, '00z4wF0iJsp6GwDkQxkGs6', 0.737, 0.818, 8.0, -2.892, 1.0, 0.0492, 0.0936, 0.0, 0.0577, 0.893, 116.93, 232859, 4), (2, '00cSN1TMyHZErfyJbLMB05', 0.814, 0.481, 7.0, -7.892, 1.0, 0.0295, 0.345, 0.41, 0.0693, 0.834, 110.002, 208813, 4), (3, '01A7PEPSnmtixFPfB2UTal', 0.793, 0.631, 11.0, -6.109, 0.0, 0.0998, 0.0406, 0.000478, 0.207, 0.231, 119.971, 226693, 4), (4, '00BuKLSAFkaEkaVAgIMbeA', 0.825, 0.832, 5.0, -5.853, 0.0, 0.0403, 0.00587, 0.000789, 0.114, 0.713, 122.021, 220627, 4), (5, '00rrluZUPNbfTSWvodUZbV', 0.513, 0.546, 5.0, -5.703, 1.0, 0.0282, 0.0181, 0.0, 0.122, 0.245, 141.876, 246560, 4), (6, '00qOE7OjRl0BpYiCiweZB2', 0.357, 0.653, 9.0, -5.554, 1.0, 0.0654, 0.0828, 0.0, 0.0844, 0.522, 176.647, 259800, 4), (7, '00LfFm08VWeZwB0Zlm24AT', 0.662, 0.748, 5.0, -3.041, 0.0, 0.268, 0.688, 8.43e-06, 0.0841, 0.535, 82.331, 239027, 4), (8, '019FM2BxcPE2vyGWeOWhvS', 0.409, 0.602, 0.0, -7.025, 0.0, 0.108, 0.0974, 0.0, 0.107, 0.186, 110.463, 243493, 4), (9, '01KdLUbWrTXdviMq7

#### get_table_df(conn, table_name, limit)

The last function simply returns the data in our tables as a pandas DataFrame for use by our recommendation algorithm. It takes in the db connection object, the table we want to export, and the limit or number of rows we want to populate into our DataFrame (string parameter since we want to put it in the sql query).

In [8]:
features_df = get_table_df(conn, 'features', '10')
features_df

INFO:root:Reading table features from database.


,track_id,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1,00z4wF0iJsp6GwDkQxkGs6,0.737,0.818,8.0,-2.892,1.0,0.0492,0.09360,0.000000,0.0577,0.893,116.930,232859,4
1,2,00cSN1TMyHZErfyJbLMB05,0.814,0.481,7.0,-7.892,1.0,0.0295,0.34500,0.410000,0.0693,0.834,110.002,208813,4
2,3,01A7PEPSnmtixFPfB2UTal,0.793,0.631,11.0,-6.109,0.0,0.0998,0.04060,0.000478,0.2070,0.231,119.971,226693,4
3,4,00BuKLSAFkaEkaVAgIMbeA,0.825,0.832,5.0,-5.853,0.0,0.0403,0.00587,0.000789,0.1140,0.713,122.021,220627,4
4,5,00rrluZUPNbfTSWvodUZbV,0.513,0.546,5.0,-5.703,1.0,0.0282,0.01810,0.000000,0.1220,0.245,141.876,246560,4
5,6,00qOE7OjRl0BpYiCiweZB2,0.357,0.653,9.0,-5.554,1.0,0.0654,0.08280,0.000000,0.0844,0.522,176.647,259800,4
6,7,00LfFm08VWeZwB0Zlm24AT,0.662,0.748,5.0,-3.041,0.0,0.2680,0.68800,0.000008,0.0841,0.535,82.331,239027,4
7,8,019FM2BxcPE2vyGWeOWhvS,0.409,0.602,0.0,-7.025,0.0,0.1080,0.09740,0.000000,0.1070,0.186,110.463,243493,4
8,9,01KdLUbWrTXdviMq7bDhip,0.672,0.711,1.0,-5.492,0.0,0.3370,0.07930,0.000000,0.7600,0.518,140.058,261213,4
9,10,00pgvR1zUVYubZpdY7jryZ,0.681,0.618,1.0,-7.952,0.0,0.1080,0.06300,0.000063,0.0451,0.167,129.823,108760,4


### References

1. [Spotify Million Playlist Dataset](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge)

2. [@nsanka Github Reference](https://github.com/nsanka/RecSys/blob/main/code/read_spotify_million_playlists.py)

3. [SQLite3 Documentation](https://docs.python.org/3/library/sqlite3.html)

4. [SQLite3 Website](https://www.sqlite.org)